<a href="https://colab.research.google.com/github/ranggacey/fuzzy/blob/main/Sugeno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy scikit-fuzzy matplotlib ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 920.8/920.8 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 57.6 MB/s eta 0:00:00


In [2]:
import numpy as np
import skfuzzy as fuzz
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider

# ==========================================
# 1. Domain Variabel [cite: 21-24]
# ==========================================
x_suhu = np.arange(0, 101, 1)
x_tekanan = np.arange(0, 11, 1)

# Fungsi keanggotaan (Suhu) [cite: 26-36]
suhu_rendah = fuzz.trimf(x_suhu, [0, 0, 40])
suhu_sedang = fuzz.trimf(x_suhu, [30, 50, 70])
suhu_tinggi = fuzz.trimf(x_suhu, [60, 100, 100])

# Fungsi keanggotaan (Tekanan) [cite: 37-48]
tekanan_rendah = fuzz.trimf(x_tekanan, [0, 0, 4])
tekanan_sedang = fuzz.trimf(x_tekanan, [3, 5, 7])
tekanan_tinggi = fuzz.trimf(x_tekanan, [6, 10, 10])

# ==========================================
# 2. Fungsi Sugeno [cite: 50-109]
# ==========================================
def sugeno(suhu_input, tekanan_input):
    # --- Fuzzifikasi ---
    # Menggunakan prefix 'u_' untuk derajat keanggotaan (membership degree)
    u_s_rendah = fuzz.interp_membership(x_suhu, suhu_rendah, suhu_input)
    u_s_sedang = fuzz.interp_membership(x_suhu, suhu_sedang, suhu_input)
    u_s_tinggi = fuzz.interp_membership(x_suhu, suhu_tinggi, suhu_input)

    u_t_rendah = fuzz.interp_membership(x_tekanan, tekanan_rendah, tekanan_input)
    u_t_sedang = fuzz.interp_membership(x_tekanan, tekanan_sedang, tekanan_input)
    u_t_tinggi = fuzz.interp_membership(x_tekanan, tekanan_tinggi, tekanan_input)

    # --- RULES (Inferensi) ---

    # Rule 1: IF suhu rendah AND tekanan rendah THEN risiko = 10 [cite: 71-73]
    w1 = np.fmin(u_s_rendah, u_t_rendah)
    z1 = 10

    # Rule 2: IF suhu sedang AND tekanan sedang THEN risiko = 50 [cite: 77-79]
    w2 = np.fmin(u_s_sedang, u_t_sedang)
    z2 = 50

    # Rule 3: IF suhu tinggi OR tekanan tinggi THEN risiko = 90 [cite: 84-85]
    # Note: Operator OR menggunakan np.fmax
    w3 = np.fmax(u_s_tinggi, u_t_tinggi)
    z3 = 90

    # Rule 4: IF suhu tinggi AND tekanan rendah THEN risiko = 60 [cite: 89-90]
    w4 = np.fmin(u_s_tinggi, u_t_rendah)
    z4 = 60

    # --- Defuzzifikasi (Weighted Average) --- [cite: 97-106]
    w = np.array([w1, w2, w3, w4])
    z = np.array([z1, z2, z3, z4])

    # Mencegah division by zero jika tidak ada rule yang aktif
    if np.sum(w) == 0:
        result = 0
    else:
        result = np.sum(w * z) / np.sum(w)

    return result

# ==========================================
# 3. Widget Interaktif & Visualisasi [cite: 111-162]
# ==========================================
def interactive_sugeno(suhu, tekanan):
    hasil = sugeno(suhu, tekanan)
    print(f"Hasil Prediksi Risiko (Sugeno): {hasil:.2f}")

    # Setup Plot
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4))

    # Visualisasi Suhu
    ax1.plot(x_suhu, suhu_rendah, label="Rendah")
    ax1.plot(x_suhu, suhu_sedang, label="Sedang")
    ax1.plot(x_suhu, suhu_tinggi, label="Tinggi")
    ax1.axvline(suhu, color='red', linestyle='--', label='Input')
    ax1.set_title("Fungsi Keanggotaan Suhu")
    ax1.legend()

    # Visualisasi Tekanan
    ax2.plot(x_tekanan, tekanan_rendah, label="Rendah")
    ax2.plot(x_tekanan, tekanan_sedang, label="Sedang")
    ax2.plot(x_tekanan, tekanan_tinggi, label="Tinggi")
    ax2.axvline(tekanan, color='red', linestyle='--', label='Input')
    ax2.set_title("Fungsi Keanggotaan Tekanan")
    ax2.legend()

    plt.show()

# Jalankan Slider
interact(
    interactive_sugeno,
    suhu=FloatSlider(min=0, max=100, step=1, value=34),     # Value default disesuaikan dgn contoh PDF
    tekanan=FloatSlider(min=0, max=10, step=0.5, value=5)
)

interactive(children=(FloatSlider(value=34.0, description='suhu', step=1.0), FloatSlider(value=5.0, descriptio…

<function __main__.interactive_sugeno(suhu, tekanan)>